# About this Page
See the slack **#ts-logging** channel for discussion about this page.  There you can report problems, questions, and requests for changes.  *What are we missing?*

## What is new in this app?(newest change at top of list)
- Do not show Time Loss info unless it is greater than zero
- Do not show software or hardware components (they are rarely used anyhow)
- <font style="background-color:#ffff99">Embedded comments and questions to reader (Yellow background)</font>
- This page tries to work around existing problems with APIs used for source data. (There is a ticket to fix the APIs)
- <font style="background-color:red; color:#90EE90">Errors from APIs are displayed in this page (Red background).</font>


In [1]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "TODAY"  # TODO Change to 'YESTERDAY' and 'TODAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

verbose = "false"  # TODO change to false before push, else true

In [2]:
# IMPORT everything
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        'TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP.'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    dev_msg = 'Imported lsst.ts.logging_and_reporting from local packages.'
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [3]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", 
    ut.Server.usdf) # TODO try with "usdf" before push (also "summit")

In [4]:
# Validate parameters, return usable ones
usable, error = ut.fallback_parameters(day_obs, number_of_days, verbose)
if error:
    print(error)
print(usable, error)

date = ut.get_datetime_from_dayobs_str(usable['day_obs'])
# date:  is EXCLUSIVE (upto, but not including)
days = usable['number_of_days']
verbose = usable['verbose']

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

{'day_obs': 'TODAY', 'number_of_days': 1, 'verbose': False} 


In [5]:
# Read records from (almost) all sources
timer = ut.Timer()
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,                
    limit=5000,
    exclude_instruments = []      # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

----------

In [6]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(f"# Showing data for {min_date.date()} to {max_date.date()} for {instrum_str}")
md(f"Most recently run on: **{dt.datetime.now().replace(microsecond=0).isoformat(sep=' ')}**")

# Showing data for 2024-11-21 to 2024-11-22 for LSSTComCamSim, LATISS, LSSTComCam, LSSTCam

Most recently run on: **2024-11-21 23:57:27**

# Table of Contents
* [Night Report](#Night-Report-BETA)
    - AuxTel
    - Simonyi
* [Almanac](#Almanac-BETA)
* [Summary plots of whole night](#Summary-plots-of-whole-night-DRAFT)
* [Time Accounting](#Time-Accounting-DRAFT)
* [Jira Tickets](#Jira-Tickets-BETA)
    - AuxTel
    - Simonyi
* [BLOCKS Observed](#BLOCKS-Observed-DRAFT)
    - AuxTel
    - Simonyi
* [Data Log](#Data-Log-BETA)
* [Narrative Log](#Narrative-Log-BETA)
* [Developer Only Section](#Developer-Only-Section-REMOVE)

## Night Report <font style="background-color:green; color:white; font-size:20px">BETA</font> 
<font style="background-color:#ffff99">Possible Changes</font>

- Word wrap instead of chopping words at end of line?
- (Add summary from AI)

In [7]:
# Night Report
# Display time log
allrep.nig_rep.time_log_as_markdown()

### Telescope: Simonyi

```
====M1M3 work log==
Bump test done around 16:48pm CLT/19:48pm UTC, 104, 227, 409 was failed initially. However, 104 and 227 passed after the second attempt while FA449 is failed with 2nd attempt. Petr mentioned that 409 is marginal but still fine. 

=== Daytime work log===
We started warmup hexapod around 14pmCLT/17pm UTC, however we had an issue with warmup hexapod scripts. Hexapods moved without issues with EUI and run_command.py. Around 14:43pm CL/17:43pm UTC, we had an issue with GIS, so M1M3, Hexapods in fault (OBS-701). It was cleared then we closed the mirror cover for dome handover. Dome handover was completed without any issue at 15:29 pm UTC/18:29 pm UTC. We found that the even after moving hexapod with run_command.py inPosition for both hexapods seem not working. This can be a blocker for night operations, we pinged people in charge. 17:30pm/20:30 pm, Te-wei roll-backed the fix happened earlier today (see deatils in OBS-702). Then warming-up hexapods worked well without an issue. At 17:53pm CLT/20:53pm CLT we turned on LaserTracker to check it worked mostly fine except one MTPtg fault while tracking. Comcam checkout, Simonyi on Sky checkout (BLOCK-T245), and M2 and Camera alignment (BLOCK-T88) are all done. 20:28 pm CLT/23:23 pm UTC, we had an issue with GIS heartbeat flicker and almost all system were fault due to hard stop engaged (OBS-701). 



We started the TMA checkout around 19:15pm and all passed except that a known issue with turning on the elevation. 

Also Chris S. and Paulo L. joined tonight's operation and thanks for supports from Tiago R, Te-wei T, and others.
```

Telescope Status: 

*Authors: Kevin Fanning, Yijung Kang, Karla Pena, Paulina Venegas, Bryce Kalmbach, Andrew Connolly, Parker Fagrelius, Bruno Quint, Keith Bechtol*

## Almanac <font style="background-color:green; color:white; font-size:20px">BETA</font> 
<font style="background-color:#ffff99">
Add column for Chile timezone?
</font>

In [8]:
# Display various almanac values (for moon, sun)
#!rep.AlmanacReport().day_obs_report(min_day_obs)
allrep.alm_rep.day_obs_report()

**Almanac for the observing night starting 2024-11-21**

Sun Set,2024-11-21 23:18:34,
Evening Civil Twilight,2024-11-21 23:49:23,(-6 degrees)
Evening Nautical Twilight,2024-11-22 00:21:27,(-12 degrees)
Evening Astronomical Twilight,2024-11-22 00:55:19,(-18 degrees)
Solar Midnight,2024-11-22 04:29:01,
Moon Rise,2024-11-22 05:01:56,
Morning Astronomical Twilight,2024-11-22 08:02:43,(-18 degrees)
Morning Nautical Twilight,2024-11-22 08:36:35,(-12 degrees)
Morning Civil Twilight,2024-11-22 09:08:39,(-6 degrees)
Sun Rise,2024-11-22 09:39:28,
Moon Set,2024-11-22 15:51:00,


## Summary plots of whole night <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>

(content not yet defined in storyboard)

<font style="background-color:#ffff99">Possible features</font>

- plot_observation_gap_detail

In [9]:
# Plot Observation (Exposure) gaps
# rollup,detail = allsrc.get_observation_gaps()
# allrep.plot_observation_gap_detail(detail)

## DDV <font style="background-color:green; color:white; font-size:20px">BETA</font>

In [10]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

Access DDV part of RubinTV: https://usdf-rsp-dev.slac.stanford.edu/rubintv/ddv/index.html

## Time Accounting <font style="background-color:green; color:white; font-size:20px">BETA</font>
TODO:
- Time-loss for Fault and Weather  (very rarely in narrativelog/messages)
- SlewTime from TMAEvent (or similar)

SlewTime (and probably others) from EFD are only EXPECTED (topic="lsst.sal.Scheduler.logevent_target"), not ACTUAL.
To get ACTUAL time, we need to use something like 
[**TMAEvent**](https://github.com/lsst-sitcom/summit_utils/blob/f901ca4943c082500a1f357eb070a4971925c476/python/lsst/summit/utils/tmaUtils.py#L1195) 
to combine multiple real events into a synthetic event.  See DM-46102.


In [11]:
# Exposure Tally
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

,LSSTComCam
Total Night,7:07:23
Total Exposure,0:00:16
Slew time(1),0:00:00
Readout time(2),0:00:31
Time loss to fault,0
Time loss to weather,0
Idle time,7:06:36
Number of exposures,13
Mean readout time,0:00:02
Number of slews(1),0


- (1) *Expected* slew time as per Schedular
- (2) There is no practical way to get detector read-out time.  A value of 2.3 seconds was inferred from experiments.

## Jira Tickets <font style="background-color:green; color:white; font-size:20px">BETA</font> 

<font style="background-color:#ffff99">Possible Changes</font>

- Observing Operations - OBS
- TODO: better tickets using Jira API to get selected fields (which) from tickets

In [12]:
if allsrc.urls:
    for url in allsrc.urls:
        md(f"- {mdpathlink(url)}")

- [obs-662](https://rubinobs.atlassian.net/browse/obs-662)

- [OBS-701](https://rubinobs.atlassian.net/browse/OBS-701)

- [obs-653](https://rubinobs.atlassian.net/browse/obs-653)

- [OBS-693](https://rubinobs.atlassian.net/browse/OBS-693)

- [OBS-699](https://rubinobs.atlassian.net/browse/OBS-699)

- [obs-701](https://rubinobs.atlassian.net/browse/obs-701)

- [OBS-698](https://rubinobs.atlassian.net/browse/OBS-698)

- [OBS-700](https://rubinobs.atlassian.net/browse/OBS-700)

In [13]:
# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    endpoint_url = allsrc.nig_src.status["reports"]["endpoint_url"]
    md(f"No tickets found using: [API Data]({endpoint_url}) in `confluence_url`.")

No tickets found using: [API Data](https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241121&max_day_obs=20241122) in `confluence_url`.

## BLOCKS Observed <font style="background-color:yellow; color:black; font-size:20px">DRAFT</font>
- TODO: BLOCKS observed

## Data Log <font style="background-color:green; color:white; font-size:20px">BETA</font>

In [14]:
# Exposure Report
for instrum in allsrc.exp_src.exposures.keys():
    md(f'**{instrum}: {len(allsrc.exp_src.exposures[instrum])} total exposures**')
    df_dict = allsrc.fields_count_exposure(instrum)
    for field_name,df in df_dict.items():
        if df.empty:
            continue
        print(field_name.title().replace('_',' '))
        display(HTML(df.style.hide().to_html(escape=False)))

**LSSTComCamSim: 0 total exposures**

**LATISS: 0 total exposures**

**LSSTComCam: 13 total exposures**

Observation Type


Detail,good,questionable,junk,unknown
acq,0,0,0,3
bias,0,0,0,10


Observation Reason


Detail,good,questionable,junk,unknown
comcam-usdf-checkout,0,0,0,10
tmacheckout,0,0,0,3


Science Program


Detail,good,questionable,junk,unknown
BLOCK-T246,0,0,0,10
BLOCK-T250,0,0,0,3


**LSSTCam: 0 total exposures**

## Narrative Log <font style="background-color:green; color:white; font-size:20px">BETA</font> 

<font style="background-color:#ffff99">Possible changes</font>

- Group messages by hour? Might be great when there many simple messages minutes apart. Less so when tracebacks are included.
- Truncate text messages to N chars (ending with "..." as link if truncated, link goes to page with full message record including text and all attributes)
- Detect some kinds of "blocks" and render in smaller text (+ other format changes)

In [15]:
# Narrative Report
# Time Log
allrep.nar_rep.time_log_as_markdown()



h3. Error/Failure explanation:
We had failures of the Camera cable wrap, axes timeouts, and frequent communications errors that impacted our control software. We lost over 2 hours of sky time struggling with these issues. 
h3. What were you doing when the failure occurred?:
Tonight we had several more instances of control system crashes due to timeouts. In addition to these incidents, the CCW would not follow the rotator, stall out during short or tracking movements, and then trigger a fault in the rotator following. We also had several recurring axes timeouts that required us to attempt initializing our control software several times before we could move. 
h3. Detailed description of the steps followed that caused the error/failure:
Here is a list of some timestamps that indicate the approximate times of communications failures. These demonstrate startTracking command timeouts, CCW timeouts, and axes timeouts. 

Example 1:
2024/11/21 02:11:02 TAI

Error MTMount replied error for startTracking command (1732828411) Error no:1 message:Failed: Command with id 447721 has timed 

Example 2: 04:32:41
The cable cable wrap faulted the MTMount. The rotator was not commanded anywhere, but the CCW just decided to quit. 

Example 3:
2024/11/21 11:02:09 TAI


Command BothAxesResetAlarm(sequence_id=1036635, command_code=<CommandCode.BOTH_AXES_RESET_ALARM: 37>, source=<Source.CSC: 1>, timestamp=1732186929.5137012) failed, waiting 60s and retrying.

<pre style='color: black; background-color: #FFDDDD; font-size: 0.875em; margin-left: 20px; '>Traceback (most recent call last):

  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/site-packages/lsst/ts/mtmount/mtmount_csc.py&quot;, line 886, in enable_devices

    await self.send_command(command, do_lock=True, timeout=timeout)

  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/site-packages/lsst/ts/mtmount/mtmount_csc.py&quot;, line 1173, in send_command

    return await self._basic_send_command(command=command, timeout=timeout)

           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/site-packages/lsst/ts/mtmount/mtmount_csc.py&quot;, line 1259, in _basic_send_command

    await asyncio.wait_for(

  File &quot;/opt/lsst/software/stack/miniconda/lib/python3.11/asyncio/tasks.py&quot;, line 489, in wait_for

    return fut.result()

           ^^^^^^^^^^^^

lsst.ts.salobj.base.ExpectedError: Command with id 1036635 has timed out after not receiving finished</pre>


- Link: [OBS-693](https://rubinobs.atlassian.net/browse/OBS-693)

- **2024-11-21 11:30**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Moving to high elevation to open mirror covers as requested for the daytime. 


Lol. JK. We didn't do that, because it was likely an old request.</pre>


- **2024-11-21 11:06**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Moving TMA in elevation.</pre>


- **2024-11-21 11:05**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Recovered telescope to perform BLOCK-240.</pre>


...

- **2024-11-21 16:54**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Start Powering on the Camera Rotator.</pre>


- **2024-11-21 16:08**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Done for Comcam checkout. Rubin TV and CCS are showing the image</pre>


- **2024-11-21 16:04**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>BLOCK-T246 Comcam USDF checkout running. Checking Comcam is ready for today's operation.</pre>


- **2024-11-21 16:02**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Around 8-9pm, 9.8C is expected. Current set point is 8C so we are good</pre>


- **2024-11-21 15:08**   **MainTel** 

<pre style='color: black; background-color: white; font-size: 1em; margin-left: 0px; '>Daytime observers start shift.</pre>


-----------
------------

# Developer Only Section <font style="background-color:red; olor:black; font-size:20px">REMOVE</font>
Contains stuff only expected to be useful to developers.

This may also contain sections that have moved out of the user section because they are no longer defined in the Storyboard.

## Overview 


In [16]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
""")

## Project-Wide Night(s) Report 
- Run on logs and databases from **https://usdf-rsp-dev.slac.stanford.edu/**
- Report **1 observing night(s)** with the last reported night starting on **2024-11-21 12:00:00**.
- min_dayobs='2024-11-21', max_dayobs='2024-11-22'
- Using ***Prototype* Logging and Reporting** Version: **52.dev0+g783922c.d20241115**
- have_consdb = False
- Imported lsst.ts.logging_and_reporting from local packages.
- allsrc.nig_src.timeout=(5.05, 20.0)


## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [17]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(
    index=False, 
    render_links=True,
    escape=False)
display(HTML(table))

Endpoint,Records,URL
nightreport/reports,1,https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241121&max_day_obs=20241122
exposurelog/instruments,3,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/instruments
exposurelog/exposures.LSSTComCamSim,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCamSim&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241121&max_day_obs=20241122
exposurelog/exposures.LATISS,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LATISS&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241121&max_day_obs=20241122
exposurelog/exposures.LSSTComCam,13,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTComCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241121&max_day_obs=20241122
exposurelog/exposures.LSSTCam,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/exposures?registry=1&instrument=LSSTCam&order_by=-timespan_end&offset=0&limit=2500&min_day_obs=20241121&max_day_obs=20241122
exposurelog/messages,0,https://usdf-rsp-dev.slac.stanford.edu/exposurelog/messages?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=2500&min_day_obs=20241121&max_day_obs=20241122
narrativelog/messages,166,https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-21T00%3A00%3A00&max_date_added=2024-11-22T00%3A00%3A00
efd/targets,0,NA


## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [18]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

### Overview for Service: `nightreport` [1]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/nightreport/reports?is_human=either&is_valid=true&order_by=-day_obs&offset=0&limit=100&min_day_obs=20241121&max_day_obs=20241122

None
- Got 1 records.  


### Overview for Service: `exposurelog` [0]

...

None
- Got 0 records.  


### Overview for Service: `narrativelog` [166]

None


- Endpoint: https://usdf-rsp-dev.slac.stanford.edu/narrativelog/messages?is_human=either&is_valid=true&order_by=-date_added&offset=0&limit=1000&min_date_added=2024-11-21T00%3A00%3A00&max_date_added=2024-11-22T00%3A00%3A00

None
- Got 166 records.  


In [19]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

In [20]:
# Time Log
#! allrep.exp_rep.time_log_as_markdown()

## Finale

In [21]:
print(f'Elapsed time (excluding code import): {timer.toc:.1f} seconds')
print(f"Finished {str(dt.datetime.now().replace(microsecond=0))} UTC")

Elapsed time (excluding code import): 4.8 seconds
Finished 2024-11-21 23:57:28 UTC
